In [1]:
import pandas as pd
x = pd.read_csv("data/13-x.csv", header=None).to_numpy()
y = pd.read_csv("data/13-y.csv", header=None).to_numpy()

In [2]:
# create dataset using torch
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor 
class NewDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.from_numpy(x).float() 
        self.y = torch.from_numpy(y).long().squeeze()

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [3]:
def train(model, loader, criterion, device):
    for i, (x,y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        # reset the optimizer 
        optimizer.zero_grad()
        logit = model(x)
        loss = criterion(logit, y)
        # backward for calculate gradients
        loss.backward()
        # update the model using the gradient
        optimizer.step() 

def eval(model, loader, device):
    y_pred = []
    y_true = []
    # don't calculate gradient in evaluation
    with torch.no_grad():
        for i, (x,y) in enumerate(loader):
            x, y = x.to(device), y.to(device)
            logit = model(x)
            pred = torch.argmax(logit, dim=-1)
            y_pred.append(pred)
            y_true.append(y) 
            
    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)
    # calculate accuracy
    return (y_pred == y_true).float().mean().item()

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# create dataset using torch 
dataset = NewDataset(x=x,y=y)

# split dataset
size= [int(len(dataset)*ratio) for ratio in [0.7, 0.2, 0.1]]
datasets = torch.utils.data.random_split(dataset, size)

# create dataloader
loaders = [DataLoader(d, batch_size=128, shuffle=(i==0), num_workers=0, pin_memory=False) 
           for i, d in enumerate(datasets)]

# use gpu or not 
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

# create model
num_features = x.shape[-1]
num_classes = 10
model = nn.Sequential(
    nn.Linear(x.shape[-1], 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, num_classes)
).to(device)

# create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# train 
max_epoch = 100 
best_val_acc = 0

# train 
max_epoch = 100 
best_val_acc = 0
PATH = 'checkpoint.pt'
for epoch in range(max_epoch):
    # train 
    train(model, loaders[0], criterion, device)
        
    # evaluation over validation
    val_acc = eval(model, loaders[1], device)
    if best_val_acc < val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), PATH)

    
    print(f'Epoch: {epoch},  Valiadtion accuracy {val_acc}')

# evaluation over test
model.load_state_dict(torch.load(PATH))
val_acc = eval(model, loaders[1], device)
test_acc = eval(model, loaders[2], device)
print(f'Test accuracy {test_acc}, Valiadtion accuracy {val_acc}')

Epoch: 0,  Valiadtion accuracy 0.44999998807907104
Epoch: 1,  Valiadtion accuracy 0.5490000247955322
Epoch: 2,  Valiadtion accuracy 0.5839999914169312
Epoch: 3,  Valiadtion accuracy 0.6380000114440918
Epoch: 4,  Valiadtion accuracy 0.6520000100135803
Epoch: 5,  Valiadtion accuracy 0.6869999766349792
Epoch: 6,  Valiadtion accuracy 0.6980000138282776
Epoch: 7,  Valiadtion accuracy 0.7120000123977661
Epoch: 8,  Valiadtion accuracy 0.7239999771118164
Epoch: 9,  Valiadtion accuracy 0.7590000033378601
Epoch: 10,  Valiadtion accuracy 0.7699999809265137
Epoch: 11,  Valiadtion accuracy 0.7799999713897705
Epoch: 12,  Valiadtion accuracy 0.7940000295639038
Epoch: 13,  Valiadtion accuracy 0.7889999747276306
Epoch: 14,  Valiadtion accuracy 0.796999990940094
Epoch: 15,  Valiadtion accuracy 0.8029999732971191
Epoch: 16,  Valiadtion accuracy 0.8230000138282776
Epoch: 17,  Valiadtion accuracy 0.8109999895095825
Epoch: 18,  Valiadtion accuracy 0.8180000185966492
Epoch: 19,  Valiadtion accuracy 0.8159999